In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import catboost as ctb
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
import category_encoders as ce

In [27]:
train_set = pd.read_csv('train_prepared.csv', encoding='utf-8')
test_set = pd.read_csv('test_prepared.csv', encoding='utf-8')

In [28]:
train_set.head()

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,Quote_Type,Target,Convertibility,Total_Amount_Sum_USD,Total_Taxable_Amount_USD,...,Opportunity_Type_target,Source _target,Region_target,Or,Expiry_Days_Create,Expiry_Days_Modified,Days_Modified_Diff_to_Start_Delivery,Days_Modified_Diff_to_End_Delivery,Quarter_Creation,Quarter_Delivery
0,1,1,1,1,0,1,0,1.131096,5964043.80,5964043.80,...,0.517054,0.509776,0.462820,0,54.0,-135.0,43.0,-17.0,4,2
1,0,0,0,0,1,1,1,1.131094,54552.68,54552.68,...,0.517054,0.509776,0.462820,0,44.0,5.0,-3.0,-5.0,4,1
2,0,0,0,0,2,1,1,1.000000,83865.60,83865.60,...,0.517054,0.438859,0.444896,0,297.0,1.0,248.0,248.0,4,1
3,1,0,1,0,3,1,0,1.000000,7421881.50,7421881.50,...,0.156954,0.632822,0.444896,1,NaN,NaN,54.0,-4.0,4,1
4,1,0,1,0,4,1,0,1.000000,13357192.50,13357192.50,...,0.156954,0.632822,0.444896,1,542.0,-298.0,54.0,27.0,4,1


In [29]:
X = train_set.drop(columns=['Target', 'Opportunity_ID'])
y = train_set.Target

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

cat = ctb.CatBoostClassifier(silent=True)

cat.fit(X_train, y_train)

In [31]:
f_imp = cat.feature_importances_
f = X_test.columns
fts = ['Opportunity_ID', 'Target']

for i in range (len(f_imp)):
    print("{: >10}\t\t{: >50}".format(f[i],f_imp[i]))
    if f_imp[i]>=1:
        fts.append(f[i])

Pricing, Delivery_Terms_Quote_Appr		                                0.2867028618194014
Pricing, Delivery_Terms_Approved		                               0.35988477524613327
Bureaucratic_Code_0_Approval		                               0.45818065470399294
Bureaucratic_Code_0_Approved		                               0.15933543717587334
Quote_Type		                                0.0374840005671818
Convertibility		                                3.4513594960735188
Total_Amount_Sum_USD		                                 1.139584145324062
Total_Taxable_Amount_USD		                                 2.415260392670484
Year_Creation		                                 1.890624360637022
Month_Creation		                                1.7675074782995006
Year_Delivery		                                 1.496142441844587
Month_Delivery		                                1.0537678839092883
Days_Passed		                                 3.837522634482917
Wait_Time_Days		                                 8.21573

In [32]:
train_filtered = train_set.loc[:, fts]
train_filtered = train_filtered.replace([np.inf,-np.inf], np.nan)

In [33]:
X = train_filtered.drop(columns=['Opportunity_ID', 'Target'])
y = train_filtered.Target

In [34]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [35]:
#params = {'depth':[3,1,2,6,4,5,7,8,9,10],
#          'iterations':[250,100,500,1000],
#          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
#          'l2_leaf_reg':[3,1,5,10,100],
#          'border_count':[32,5,10,20,50,100,200]
          #'ctr_border_count':[50,5,10,20,100,200]
#            }

#con train

#from sklearn.model_selection import RandomizedSearchCV

#cat_tuning = ctb.CatBoostClassifier(silent=True)

#random_search = RandomizedSearchCV(cat_tuning,param_distributions=params,n_iter=200, scoring='neg_log_loss',n_jobs=-1,cv=5,verbose=3)

#random_search.fit(X_train,y_train)

In [36]:
#random_search

In [37]:
test_filtered = test_set.loc[:, fts]

In [38]:
test_filtered

,Opportunity_ID,Target,Convertibility,Total_Amount_Sum_USD,Total_Taxable_Amount_USD,Year_Creation,Month_Creation,Year_Delivery,Month_Delivery,Days_Passed,...,Billing_Country_target,Delivery_Terms_target,Bureaucratic_Code_target,Opportunity_Type_target,Source _target,Region_target,Or,Expiry_Days_Create,Expiry_Days_Modified,Days_Modified_Diff_to_End_Delivery
0,10689,1,1.131098,4.155868e+05,415586.217750,2019,4,2019,5,0.0,...,0.512232,0.479354,0.593111,0.702041,0.438859,0.462820,0,18.0,18.0,-49.0
1,10690,1,1.131096,8.571262e+05,757783.500000,2019,4,2019,6,13.0,...,0.512232,0.479354,0.593111,0.702041,0.438859,0.462820,0,18.0,5.0,-64.0
2,10691,1,1.000000,2.103750e+04,189.135216,2019,4,2019,10,4.0,...,0.450638,0.543384,0.593111,0.687187,0.556701,0.444896,0,NaN,NaN,-168.0
3,10692,1,1.000000,2.169106e+06,NaN,2019,4,2019,12,4.0,...,0.450638,0.543384,0.283333,0.517054,0.632822,0.444896,0,NaN,NaN,-238.0
4,10693,1,1.000000,5.752500e+03,NaN,2019,4,2019,5,1.0,...,0.450638,0.543384,0.593111,0.687187,0.556701,0.444896,0,30.0,29.0,-19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,12364,1,1.000000,1.477500e+05,NaN,2019,4,2019,5,0.0,...,0.545454,0.527778,0.593111,0.517054,0.521311,0.462820,0,28.0,28.0,-14.0
1563,12365,1,1.000000,4.505490e+04,50961.689339,2019,4,2019,10,1.0,...,0.450638,0.543384,0.593111,0.517054,0.509776,0.444896,0,28.0,27.0,-248.0
1564,12366,1,1.000000,1.001220e+05,100122.000000,2019,4,2019,10,1.0,...,0.450638,0.543384,0.593111,0.517054,0.509776,0.444896,0,28.0,27.0,-248.0
1565,12367,0,1.000000,1.432200e+05,161995.861176,2019,4,2019,8,0.0,...,0.450638,0.543384,0.593111,0.687187,0.509776,0.444896,0,30.0,30.0,-97.0


In [39]:
#test_pred_CB = random_search.predict_proba(test_filtered.drop(columns=['Opportunity_ID', 'Target']))

In [40]:
#test_set['Pred_Tuned'] = pd.DataFrame(test_pred_CB)[1].to_list()
#test_set

In [41]:
#log_loss(test_set.Target, test_pred_CB)

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer

numeric_cols = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()

# Transformaciones para las variables numéricas
numeric_transformer = Pipeline(
                        steps=[
                            ('imputer', SimpleImputer()),
                            ('scaler', KBinsDiscretizer(n_bins=5, strategy='quantile'))
                        ]
                      )

preprocessor = ColumnTransformer(
                    transformers=[
                        ('numeric', numeric_transformer, numeric_cols)
                        ],
                    remainder='passthrough'
                    )

In [43]:
# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
best = {'learning_rate': 0.01,
 'l2_leaf_reg': 5,
 'iterations': 500,
 'depth': 7,
 'border_count': 200}

cat = ctb.CatBoostClassifier(silent=True, **best)

pipe = Pipeline([('preprocessing', preprocessor),('rf',cat)])

# Train
# ==============================================================================
# Se asigna el resultado a _ para que no se imprima por pantalla
_ = pipe.fit(X=X_train, y=y_train)

C:\Users\Francisco\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\Francisco\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\Francisco\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 11 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\Francisco\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in featur

In [44]:
log_loss(y_test, pipe.predict_proba(X_test))

0.3559440584474994

In [45]:
log_loss(test_filtered.Target, pipe.predict_proba(test_filtered.drop(columns=['Opportunity_ID', 'Target'])))

0.43236028358562856

In [46]:
best = {'learning_rate': 0.01,
 'l2_leaf_reg': 5,
 'iterations': 500,
 'depth': 7,
 'border_count': 200}

cat = ctb.CatBoostClassifier(silent=True, **best)

cat.fit(X_train, y_train)

In [47]:
log_loss(y_test, cat.predict_proba(X_test))

0.34049130546736417

In [48]:
log_loss(test_filtered.Target, cat.predict_proba(test_filtered.drop(columns=['Opportunity_ID', 'Target'])))

0.452645108391374

In [49]:
cat.predict_proba(test_set.drop(columns=['Opportunity_ID', 'Target']))

array([[0.09502041, 0.90497959],
       [0.36752237, 0.63247763],
       [0.54014642, 0.45985358],
       ...,
       [0.80575306, 0.19424694],
       [0.39362686, 0.60637314],
       [0.932212  , 0.067788  ]])

In [50]:
final = test_filtered.loc[:, ['Opportunity_ID', 'Target']].copy()
final['Target'] = pd.DataFrame(pipe.predict_proba(test_filtered.drop(columns=['Opportunity_ID', 'Target']))[:, 1])
final.to_csv('new_cat.csv', index=False)